# === Kapampangan-to-English NLLB-200 Training Pipeline (with <kap> tag) ===

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback
)
import torch
import evaluate
import numpy as np

# === 1. Config ===

In [ ]:
CSV_PATH = "data/kapampangan_english.csv"
MODEL_NAME = "facebook/nllb-200-distilled-600M"
MODEL_DIR = "./kapampangan_mt_nllb"

SPECIAL_SRC_TOKEN = "<kap>"
TGT_LANG = "eng_Latn"   # English (Latin script)

EXTRA_TOKENS = ["banua", "masanting", "eku", "ala", "nung", "manyaman", 'a', 'abagat', 'abak', 'abaka', 'abakan', 'abal', 'abala', 'abalitan', 'abalu', 'abandunadu', 'abang', 'abating', 'abayan', 'abblas', 'abbua', 'abbual', 'abbuang', 'abe', 'abeabe', 'abias', 'abisu', 'abit', 'ablad', 'ablang', 'ablas', 'abli', 'abluk', 'abo', 'abonu', 'abpa', 'abril', 'absang', 'abse', 
'absi', 'absik', 'abu', 'abugadu', 'abukadu', 'abulud', 'abusadu', 'abut', 'abyas', 'adi', 'adjar', 'admirul', 'adobu', 'adta', 'adwa', 'adwalima', 'adwang', 'adya', 'adyang', 'adyar', 'africa', 'aga', 'agad', 'agag', 'agap', 'agas', 'agawa', 'agi', 'aginaldu', 'agkas', 'agkat', 'aglo', 'agnanagnan', 'agostu', 
'agredulse', 'agta', 'agtal', 'agum', 'aguman', 'agus', 'agwanta', 'agwasinada', 'agyag', 'agyang', 'agyap', 'agyu', 'agyung', 'ah', 'aintindian', 'aintindyan', 'aisip', 'akabyusan', 'akakit', 'akala', 'akalingwan', 'akapulku', 'akarapat', 'akasya', 'akatagmu', 'akayabe', 'akbak', 'akbung', 'akikit', 'akilala', 'akilalang', 'akit', 'aklak', 'aklat', 'akmul', 'aksaya', 'aksidenti', 'aksing', 'aktang', 'aktibidad', 'aktibo', 'aku', 'akua', 'akut', 'ala', 'aladin', 'alagad', 'alaheru', 'alak', 'alakdan', 'alakton', 'alal', 'alala', 'alale', 'alambri', 'alang', 'alangalang', 'alangan', 'alarin', 'alas', 'alat', 'alata', 
'alauli', 'alaya', 'alayu', 'albe', 'alben', 'albug', 'albugan', 'albul', 'albularyu', 'albung', 'alburutu', 'alcohol', 'aldawan', 'aldo', 'aldoadlo', 'aldoaldo', 'aldoldo', 'aldong', 'alduk', 'ale', 'alertu', 'ali', 'alibub', 'alikabuk', 'alili', 'alimasag', 'alimbawa', 'alimbubu', 'alimpungatan', 'alimpuyu', 
'alingongo', 'alino', 'alipan', 'alipis', 'alipit', 'alipugpug', 'alipunga', 'alipup', 'alistu', 'alisuas', 'alisus', 'aliwa', 'aliwang', 'alkas', 'almendras', 'almuranas', 'almusal', 'alog', 'alpa', 'alpuk', 'alput', 'alsik', 'alsut', 'altar', 'alternatibu', 'alti', 'alubebe', 'aluk', 'alulud', 'alumeme', 'aluminyu', 'alun', 'alus', 'alutas', 'alwagi', 'alyad', 'ama', 'amad', 'amanu', 'amanwan', 'amasamas', 'ambag', 'ambangan', 'ambisyun', 'ambu', 'ambula', 'ambun', 'ambus', 'amil', 'amiss', 'amius', 'amiyam', 'ampang', 'ampat', 'ampil', 'ampo', 'ampon', 'ampong', 'ampulas', 'amsik', 'amu', 'amuk', 'amurikut', 'amut', 'amuyut', 'ana', 'anad', 'anak', 'anaku', 'anam', 'anao', 'anatis', 'andam', 'andaman', 'andas', 'andat', 'andyang', 'ane', 'ang', 'angab', 'ange', 'angeles', 'angga', 'anggang', 'anggi', 'anggyang', 'angin', 'angit', 'angka', 'angkad', 'angkak', 'angus', 'anidyu', 'animal', 'aninag', 'anino', 'anisadu', 'anitu', 'anta', 'antabe', 'antak', 'antala', 'anti', 'antik', 'antindyan', 'anting', 'antinganting', 'antipolu', 'anunas', 'anya', 'anyang', 'anyaya', 'anyeho', 'anyud', 'apa', 'apad', 'apag', 'apakan', 'apalit', 'apalya', 'apalyaring', 'apan', 'apanintun', 'apansinan', 'apansinang', 'apap', 'apapakayli', 'apas', 'apat', 'apayntun', 'apda', 'apendisitis', 'api', 'apia', 'apigil', 'apin', 'apir', 'apis', 'apisamutsamut', 'apitna', 'aplak', 'aplaya', 'apon', 'apsa', 'apsal', 'apsang', 'apsi', 'aptas', 'apu', 'apulid', 'apulu', 'apulung', 'apun', 'apunan', 'apungapung', 'apus', 'apyak', 'arakap', 'aral', 'arana', 'arap', 
'araramdaman', 'araro', 'aray', 'arayat', 'are', 'arekomenda', 'areni', 'areview', 'argabyadu', 'ari', 'arina', 'arinola', 'ariya', 'arkila', 'armenia', 'aro', 'arobu', 'aroy', 'arsobispu', 'aru', 'arung', 'arungarung', 'asa', 'asad', 'asado', 'asag', 'asahan', 'asal', 'asan', 'asang', 'asap', 'asawa', 'asayasa', 'asbu', 'asbuk', 'asdal', 'asdan', 'asdap', 'asias', 'asik', 'asin', 'asingku', 'asintadu', 'askad', 'askal', 'askup', 'aslag', 'aslam', 'aslat', 'asna', 'asnang', 'asne', 'asnuk', 'aspak', 'aspe', 'aspili', 'aspirin', 'asu', 'asubukan', 'asuhos', 'asuk', 'asul', 'asung', 'asungasungan', 'asus', 'aswal', 'aswas', 'aswe', 'aswiti', 'asyang', 'asyo', 'at', 'ata', 'atad', 'atap', 'atas', 'atatandanan', 'atbu', 'atbud', 'atbus', 'atdas', 'atdu', 'ate', 'atiab', 'atik', 'atin', 'ating', 'atling', 'atlu', 'atlulima', 'atlung', 'atna', 'atni', 'atsara', 'atsi', 'atsing', 'atsu', 'atul', 'atut', 'atwan', 'atyan', 'atyat', 'atyu', 'aun', 'aus', 'ausan', 'awa', 'awak', 'awakan', 'awang', 'awat', 'awig', 'awsan', 'awsang', 'awus', 'ay', 'aya', 'ayani', 'ayari', 'aydo', 'ayni', 
'ayntindyan', 'ayta', 'ayup', 'b', 'ba', 'babad', 'babagua', 'babagwa', 'babai', 'babaing', 'babala', 'babalakang', 'babalik', 'babasan', 'babatyawan', 'babi', 'babo', 'bacolor', 'baga', 'bagalan', 'bagang', 'bagbag', 'bage', 'bagebage', 'bageng', 'bagkus', 'bagok', 'bagse', 'bagsik', 'bagut', 'bagya', 'bagyu', 'bagyus', 'baha', 'baiki', 'bait', 'bakas', 'bakasyon', 'bakasyun', 'bakawan', 'bakbak', 'bakit', 'baklad', 'baklas', 'bakle', 'bakolor', 'bakting', 'bakud', 'bakuli', 'bakulud', 'bakwit', 'bakya', 'bala', 'balaba', 'balabag', 'balak', 'balaku', 'balamu', 'balanak', 'balang', 'balanga', 'balanggut', 'balangkas', 'balantakan', 'balas', 'balasenas', 'balat', 'balatkayu', 'balayan', 'bale', 'balen', 'baleng', 'balibago', 'balibagu', 'balibid', 'balibul', 'balik', 'balikat', 'balikid', 'baliktad', 'balikwas', 'balimbing', 'baling', 'balintawak', 'balisa', 'balisasa', 'balisbis', 'balisoso', 'baliswag', 'balita', 'balitang', 'baliti', 'balitkayu', 'balls', 'balobalo', 'balu', 'balubad', 'baluga', 'balugbug', 'baluk', 'balukas', 'baluktut', 'balunbalunan', 'balung', 'balungbalung', 'balungkit', 'balungkitan', 'balus', 'balwat', 'bamban', 'ban', 'bana', 'banaba', 'banak', 'banal', 'banda', 'bandang', 'bandera', 'bandeyadu', 'bandi', 'bang', 'banga', 'bangi', 'bangkabangka', 'bangkal', 'bangle', 'bangsa', 'bangungut', 'bangus', 'banlad', 'banlik', 'banlo', 'bantang', 'bantug', 'banua', 'banwa', 'banwaan', 'banwang', 'banyu', 'bapor', 'barit', 'baritan', 'barkada', 'barkun', 'baryu', 'basal', 'basan', 'basta', 'basung', 'basura', 'bat', 'bataan', 'batirul', 'batyo', 'bawas', 'bawu', 'baya', 'bayad', 'bayang', 'bayani', 'bayaran', 'bayaranan', 'bayat', 'bayo', 'bayu', 'bayung', 'bebe', 'beer', 'begukan', 'beinti', 'beintisingku', 'belita', 'bengi', 'berdi', 'besis', 'betute', 'bia', 'biabas', 'bibingka', 'bibingkaan', 'bida', 'bie', 'biernis', 'biga', 'biglang', 'bigote', 'bigoti', 'biko', 'bilao', 'bilungan', 'bingut', 'bini', 'binili', 'binite', 'binulu', 'binwat', 'binye', 'biringhi', 'biru', 
'bisa', 'bisang', 'bisikleta', 'bisita', 'biskotsu', 'biskwit', 'bistwela', 'bisugu', 'bitasang', 'bitis', 'biya', 'blender', 'bobotu', 'bola', 'bomb', 'bonsai', 'bookstore', 'bos', 'boses', 'bosis', 'boston', 'bote', 'boti', 'botika', 'botu', 'boxing', 'broccoli', 'brownies', 'buanbuan', 'bubungan', 'bucsit', 
'bukas', 'buksi', 'buku', 'bukungbukung', 'bulag', 'bulan', 'bulaon', 'bulaun', 'bulig', 'bulsa', 'bulu', 'bulung', 'bulus', 'bumbunan', 'bundagul', 'bundaki', 'bundalag', 'bunduk', 'bundukbundukan', 'bunga', 'bungal', 'buntuk', 'bunut', 'bunwan', 'bunying', 'buren', 'buri', 'buring', 'burj', 'buru', 'bus', 'busal', 'busni', 'butarul', 'butulan', 'butulbutul', 'buung', 'buysit', 'buyun', 'byahi', 'byasa', 'byasang', 'bye', 'byernis', 'cabalantian', 'cabambangan', 
'cabangcalan', 'cabetican', 'cabuluan', 'cactus', 'cafe', 'caingin', 'cakes', 'calangain', 'calantas', 'calantipe', 'calibungan', 'calibutbut', 'california', 'calinan', 'calingcuan', 'call', 'caluluan', 'calulut', 'calumpang', 'calumpit', 'camba', 'cambasi', 'camera', 'camias', 'camuning', 'canarem', 'candaba', 'candating', 'candelaria', 'cangatba', 'cansinala', 'canukil', 'capalangan', 'capas', 'capaya', 'capehan', 'carangian', 'card', 'cards', 'care', 'carmencita', 'cash', 'castillo', 'castuli', 'catmun', 'cawayan', 'cellphone', 'central', 'change', 'charger', 'ciudad', 'clark', 'clasi', 'classroom', 'climate', 'cobra', 'colgante', 'concepcion', 'consuelo', 'cookies', 'counter', 'court', 'credit', 'cruz', 'cualta', 'cuayan', 'cubcub', 'culatingan', 'culiat', 'culipat', 'culubasa', 'cupang', 'cutcut', 'cutud', 'da', 'dabulbul', 'dadalanan', 'dadalang', 'dagdag', 'dagis', 'daglus', 'dagul', 'dagus', 'daig', 'daing', 'daka', 'dakal', 'dakami', 'dakang', 'dakap', 'dakayu', 'dake', 'dakeng', 'dakit', 'dakitan', 'daklung', 'daklut', 'daku', 'dakung', 'dakurak', 'dala', 'dalag', 'dalaga', 'dalagita', 'dalakit', 'dalan', 'dalandan', 'dalangat', 'dalas', 'dalayap', 'daldak', 'daldal', 'dali', 'dalig', 'dalit', 'dalita', 'dalo', 'dalpak', 'dalukduk', 'dalum', 'dalumdum', 'dalung', 'dalungdung', 'dalusdus', 'dalusung', 'dalyawat', 'damas', 'dambuwala', 'damdam', 'damdaman', 'damput', 'damuku', 'damukus', 'damut', 'dana', 'danak', 'danakang', 'dane', 'dang', 'dangal', 'dangan', 'dangat', 'danom', 'danum', 'danun', 'danup', 'danupan', 'dapal', 'dapat', 'dapilus', 'dapot', 'daptan', 'dapuk', 'dapurak', 'dara', 'darak', 'darakal', 'daralan', 'daramdaman', 'daras', 'daratang', 'dareng', 'daretsu', 'dase', 'datang', 'date', 'datun', 'dau', 'daun', 'dawak', 'day', 'daya', 'dayami', 'dayas', 'dayatmalat', 'dayi', 'dayu', 'dayuput', 'de', 'dedikasyun', 'dekap', 'dela', 'delikadu', 'delikdu', 'den', 'denas', 'deng', 'deni', 'deta', 'detalye', 'detalyi', 'di', 'dies', 'dikdik', 'dikil', 'dikit', 'diksyunaryo', 'diksyunaryu', 'dikulor', 'dikut', 'dila', 'dilat', 'dilensya', 'dili', 'dilo', 'dilu', 'dilukan', 'dimdam', 'dimla', 'dimut', 'dinalan', 'dinan', 'dinat', 'dinatang', 'dine', 'ding', 'dingding', 'dininan', 'dinusug', 'diolen', 'dios', 'diperensya', 'direkta', 'diretsu', 'diri', 'diris', 'dirit', 'diritak', 'dirya', 'disan', 'disgrasya', 'disgrasyada', 'disisyun', 'diskarti', 'dispu', 'disyertu', 'ditak', 'diwa', 'diwal', 'diwata', 'diyang', 'doctor', 'doktor', 'donat', 'dong', 'dos', 'dosi', 'drills', 'drinks', 'duat', 'dugdug', 'dugsung', 'dugtung', 'dugu', 'dukduk', 'duku', 'dukung', 'dulang', 'dulduk', 'duldul', 'duling', 'duluk', 'dulun', 'dulung', 'dulungan', 'dulut', 'dumalaga', 'duman', 'dumara', 'duminggu', 'dungdung', 'dungdungan', 'dungis', 'dungo', 'dungu', 'dungus', 'dungut', 'dunut', 'duru', 'durug', 'durun', 'durup', 'durut', 'dusa', 'dusdus', 'dusenang', 'dutdut', 'dutung', 'dutungamatas', 'duwi', 'duyan', 'dyip', 'dyolen', 'e', 'ebun', 'ebus', 'eganagana', 'eganaganang', 'eka', 'eke', 'eko', 'eksperimentung', 'eku', 'ekuna', 'ekupa', 'ela', 'elepanti', 'elu', 'ema', 'email', 'eme', 'emo', 'empanada', 'empleyadu', 'emu', 'emuke', 'ena', 'enaku', 'ene', 'engkantu', 'eno', 'ensaimada', 'ensalada', 'entabladu', 'entrance', 'epektu', 'era', 'eraka', 'eraku', 'eran', 'ere', 'eruplanu', 'escalator', 'eskuwela', 'eskwela', 'espasol', 'espesyal', 'estado', 'eta', 'ex', 'examination', 'exit', 'experiment', 'eya', 'eyu', 'farmacia', 'fax', 'faxardo', 'felipe', 'festival', 'fiesta', 'filipina', 'filipinu', 'film', 'fish', 'flights', 'floridablanca', 'folder', 'food', 'format', 'fountain', 'gabak', 'gabun', 'gad', 'gadya', 'gadyet', 'gaga', 'gagalgal', 'gagamit', 'gagamitan', 'gagawan', 'gagawang', 'gagayak', 'gagtu', 'gakgak', 'gala', 'galakgak', 'galal', 'galame', 'galang', 'galanggang', 'galantina', 'galasgas', 'gale', 'galgal', 'galigit', 'galimgim', 'galing', 'galingaldo', 'galo', 'galudgud', 'galukguk', 'galunggung', 'galura', 'galut', 'galyetas', 'gamal', 'gamat', 'gambul', 'gamit', 'gamitan', 'gamugamu', 'gamus', 'ganakan', 'ganap', 'gandus', 'gani', 'ganingaldo', 'gansal', 'gapang', 'gapas', 'gapuk', 'garabyadu', 'garing', 'gas', 'gasgas', 'gasolina', 'gastarul', 'gata', 'gatal', 'gatang', 'gatas', 'gatganapun', 'gatgas', 'gato', 'gatpanapun', 'gauk', 'gawa', 'gawan', 'gawang', 'gawi', 'gayak', 'gayuma', 'gcash', 'gera', 'gerona', 'gewa', 'giam', 'gigut', 'gigutan', 'gilagid', 'gili', 'gilid', 'giling', 'gilingan', 'gimik', 'ginaga', 'ginalikgik', 'ginamit', 'ginauk', 'ginawa', 'ginawuk', 'giniling', 'ginising', 'gintu', 'ginu', 'ginulisak', 'gisa', 'gisadu', 'gisan', 'gisi', 'gising', 'gisingan', 'gisyan', 'gitara', 'gogo', 'grecia', 'grocery', 'grupu', 'guagua', 'gubat', 'gubyernu', 'gugu', 'gugul', 'gulaman', 'gule', 'gulis', 'gulisak', 'gulu', 'gulut', 'gumamela', 'gumamit', 'gumastus', 'gunam', 'gunamgunam', 'gunting', 'gupu', 'gusi', 'gutli', 'guyabnan', 'guyud', 'gyang', 'gym', 'haji', 'halal', 'handball', 'handles', 'hapihapi', 'hapon', 'hasahasa', 'headphone', 'helicopter', 'helmet', 'hepe', 'high', 'holidays', 'hotel', 'huwebis', 'hyah', 'i', 
'ibat', 'ibatan', 'ibie', 'ibpa', 'ibug', 'ibusni', 'ibwat', 'ibye', 'icharge', 'ideya', 'iemail', 'igit', 'igpa', 'igu', 'igut', 'ika', 'ikami', 'ikat', 'iket', 'iki', 'ikit', 'ikutang', 'ikwa', 'ikwang', 'ilap', 'ilib', 'iligtas', 'ilis', 'ilo', 'ilug', 'ima', 'imalan', 'imalang', 'imalid', 'imbentuan', 'imbut', 'imik', 'impaktu', 'impis', 'impormasyun', 'importanti', 'imposibli', 'impusibli', 'imu', 'imulagat', 'imut', 'ina', 'inabayan', 'inagkat', 'inalben', 'inamin', 'inamuyut', 'inang', 'inangan', 'inapun', 'inarap', 'inarestu', 'inaus', 'inawa', 'inda', 'indak', 'indu', 'ine', 'ing', 'ingat', 'ingatan', 'inge', 'inggil', 'inggit', 'ingit', 'ingkung', 'inglis', 'ingpis', 'ini', 'ining', 'inisip', 'inlag', 'inorder', 'inos', 'insak', 'inspirasyun', 'interbyu', 'interesadu', 'inulunan', 'inum', 'inuman', 'inut', 'inutil', 'inya', 'inyang', 'ipakit', 'ipan', 'ipanik', 'ipas', 'ipe', 'ipis', 'ipokritu', 'ipus', 'irap', 'iro', 'isa', 'isake', 'isalikut', 'isara', 'isaw', 'isding', 'isinup', 'isip', 'isipan', 'iskedyul', 'iskierda', 'iskwela', 'islam', 'isnawa', 'iso', 'isplika', 'istasyun', 'istorya', 'istu', 'istudyanti', 'istudyanting', 'istung', 'isulat', 'isumbung', 'ita', 'itad', 'italianu', 'itang', 'iti', 'itratung', 'itu', 'iwa', 'iyan', 'iyi', 'jacket', 'jalal', 'jaleya', 'jalung', 'jam', 'japan', 'jefmin', 'jet', 'jip', 'john', 'jolen', 'juliana', 'ka', 'kaba', 'kabait', 'kabaldugan', 'kabalik', 'kabalitan', 'kabalubala', 'kabalubalu', 'kaban', 'kabang', 'kabasi', 'kabaung', 'kabayangnan', 'kabayu', 'kabibi', 'kabid', 'kabilyan', 'kabirabira', 'kabisera', 'kabius', 'kabud', 'kabug', 'kabukas', 'kabulasan', 'kabundukan', 'kaburi', 'kabus', 'kabuyo', 'kabyo', 'kadayatmalatan', 'kadwa', 'kadwang', 'kagalangan', 'kagigising', 'kagiwan', 'kagrabi', 'kagulu', 'kahun', 'kai', 'kailangan', 'kaka', 'kakagising', 'kakalingwan', 'kakalingwang', 'kakaluguran', 'kakanan', 'kakandutan', 'kakarug', 'kakatmuan', 'kakawati', 'kakawe', 'kakayap', 'kakilala', 'kaku', 'kakung', 'kalabit', 'kalabkab', 'kalabukab', 'kalag', 'kalagwang', 'kalaka', 'kalakal', 'kalakalake', 'kalako', 'kalakubak', 'kalalam', 'kalamansi', 'kalamba', 'kalambat', 'kalambibit', 'kalame', 'kalamunding', 'kalamungge', 'kalang', 'kalangkang', 'kalantas', 'kalaraman', 'kalasag', 'kalaskas', 'kalasu', 'kalat', 'kalatsutsi', 'kalawang', 'kalawat', 'kalawut', 'kalbit', 'kaldereta', 'kalderu', 'kaleldo', 'kalgareta', 'kaligkig', 'kaligtasan', 'kalikubak', 'kalilipat', 'kalima', 'kaliskis', 'kalma', 'kalpinteru', 'kalubkub', 'kalubkubin', 'kalugkug', 'kaluguran', 'kaluku', 'kalukup', 'kalulu', 'kalumpang', 'kalumparit', 'kalung', 'kalupa', 'kalus', 
'kalutkut', 'kalwat', 'kalyus', 'kamaganak', 'kamal', 'kamalian', 'kamamalu', 'kamandag', 'kamangyang', 'kamantigi', 'kamatis', 'kamatsilis', 'kamau', 'kambal', 'kamera', 'kami', 'kamias', 'kamin', 'kaming', 'kamiseta', 'kamo', 'kampyun', 'kamuti', 'kanaku', 'kanakung', 'kanam', 'kanan', 'kang', 'kangkung', 'kanine', 'kanini', 'kaninu', 'kanita', 'kanite', 'kaniti', 'kaniting', 'kansasaga', 'kanta', 'kanu', 'kanung', 'kanyan', 'kanyang', 'kapa', 'kapakibatan', 'kapalangan', 'kapaligiran', 'kapamahalan', 'kapanamdaman', 'kapangan', 'kapas', 'kapat', 'kapatad', 'kapatagan', 'kapaya', 'kape', 'kapilan', 'kapitan', 'kapitangan', 'kapitna', 'kapitngan', 'kapuranggit', 'karagdagagan', 'karakal', 'karakwak', 'karaniwan', 'karanun', 'karaoke', 'karatang', 'karate', 'karekare', 
'karela', 'karelang', 'karene', 'kareng', 'karening', 'karikari', 'karin', 'karing', 'karne', 'karni', 'karulun', 'kasakit', 'kasakmal', 'kasal', 'kasalanan', 'kasalanana', 'kasalesayan', 'kasangkapan', 'kasanting', 'kasapegu', 'kasaya', 'kasi', 'kasibul', 'kasikan', 'kasilyas', 'kasim', 'kasindagul', 'kasing', 'kasingidad', 'kasingkatwa', 'kasingulaga', 'kasta', 'kasumangid', 'katalumbo', 'katamad', 'katana', 'katas', 'katawan', 'katimawan', 'katlu', 'katmu', 'katorsi', 'katsura', 'katuldwanan', 'katutuan', 'katutwan', 'katwa', 'kauran', 'kawali', 'kawangis', 'kawlan', 'kaya', 'kayabe', 'kayaldawan', 'kayanakan', 'kayang', 'kayanib', 'kayari', 'kayas', 'kaybat', 'kayi', 'kaylangan', 'kaylangang', 'kayli', 'kayu', 'kayud', 'kayumanggi', 'kayus', 'ke', 'kebaitan', 'kebaytan', 'kegisanan', 'keka', 'kekami', 'kekang', 'kekatamu', 'kekatang', 'kekayo', 'kekayu', 'kemu', 'ken', 'kendi', 'keng', 'keni', 'kening', 'kerakalan', 'keraklan', 'kesa', 'kesyap', 'keta', 'ketang', 'keti', 'keting', 'keyaryan', 'khalifa', 'kikil', 'kikislap', 'kikyak', 'kilala', 'kilayin', 'kildap', 'kilkal', 'kilo', 'kiltian', 'kilub', 'kilwal', 'kilyawan', 'kimonu', 'kinabukasan', 'kinang', 'kinangwa', 'kindergarten', 'kinera', 'king', 'kinopya', 'kinsi', 'kintal', 'kinyak', 'kiping', 'kiput', 'kirey', 'kirut', 'kisap', 'kiskis', 'kitid', 'kitkut', 'kitnan', 'klasi', 'klasing', 'ko', 'kodak', 'kodigu', 'kokomustan', 'komportabli', 'komusta', 'kong', 'kontra', 'kontrata', 'konyo', 'koreanu', 'korsunada', 'kotse', 'kotsi', 'kotsing', 'koya', 'koyoti', 'ku', 'kubeta', 'kukomustan', 'kukutang', 'kukwa', 'kula', 'kulang', 'kulatirik', 'kulebra', 'kulitis', 'kulul', 'kuma', 'kumabye', 'kumang', 'kumbinsyan', 'kumpanyang', 
'kumu', 'kuna', 'kunaman', 'kundul', 'kune', 'kung', 'kuntentu', 'kuntoru', 'kuran', 'kurangkurangan', 'kurus', 'kuryenti', 'kusina', 'kusineru', 'kutang', 
'kutnan', 'kutnanan', 'kutnangan', 'kuto', 'kutu', 'kwalta', 'kwanan', 'kwarenta', 'kwartu', 'kwartung', 'kwat', 'kwatru', 'kwekkwek', 'kwentuan', 'la', 'laar', 'labag', 'labak', 'labanan', 'labas', 'labentador', 'labi', 'labis', 'lablab', 'labor', 'laboratoryu', 'labuad', 'labug', 'labul', 'labung', 'labut', 'labuyu', 'ladlad', 'ladya', 'laga', 'lagad', 'lagapak', 'lagari', 'lagas', 'lagat', 'lagitik', 'laglag', 'lagnat', 'lagpak', 'lagu', 'lagusgus', 'lagutuk', 
'lagwa', 'lagyu', 'lain', 'lakad', 'lakanbalen', 'lakas', 'lakbang', 'lakbe', 'laklak', 'laksa', 'lakton', 'lakwan', 'lakwas', 'lakwatsa', 'lala', 'lalabangan', 'lalakad', 'lalakaran', 'lalaki', 'lalam', 'lalawe', 'lalawen', 'lalbe', 'lalik', 'lalu', 'lamak', 'laman', 'lamas', 'lamat', 'lambal', 'lambat', 'lambilambi', 'lambis', 'lambut', 'lame', 'lamesa', 'lamierda', 'lamiran', 'lamlam', 'lampin', 'lamyerda', 'lanam', 'lanat', 'lande', 'lang', 'langab', 'langi', 
'langib', 'langis', 'langitngit', 'langkap', 'langkul', 'langnis', 'lango', 'langut', 'lansangan', 'lantang', 'lantung', 'lanubo', 'lanya', 'lanyo', 'lapad', 'lapak', 'lapas', 'lapat', 'lapiris', 'lapit', 'lapitpit', 'laplap', 'lapo', 'lapulapu', 'laput', 'lapuwak', 'lara', 'laram', 'larawan', 'lare', 'lareng', 'larin', 'larinan', 'laru', 'lasa', 'lasak', 'lasing', 'lasip', 'laslas', 'laso', 'lasona', 'lasu', 'lasuk', 'lasun', 'lasut', 'late', 'lati', 'latik', 'latim', 'lauk', 'laus', 'laut', 'lawen', 'lawi', 'lawu', 'laya', 'layag', 'layas', 'layit', 'lechon', 'leko', 'lengguwahe', 'letrato', 'letratrung', 'letratu', 'letratuan', 'libri', 'libru', 'librung', 'libutad', 'ligligan', 'ligtas', 'lihim', 'likoru', 'likwan', 'lilibut', 'lima', 'limang', 'limitadu', 'limpasan', 'linakad', 'linarin', 'linawe', 'linikas', 'linub', 'linukluk', 'linundag', 'linwal', 'lisyun', 'litid', 'litratu', 'litsun', 'liualu', 'liwag', 'liwalas', 'liwas', 'liya', 'liyab', 'liyag', 'liyap', 'llintik', 'lon', 'london', 'longganiza', 'losa', 'loyalty', 'lu', 'luallungub', 'lub', 'lubao', 'lubas', 'lubid', 'lubus', 'luga', 'lugal', 'lugar', 'lugaw', 'luge', 'lugi', 'lugit', 'luglug', 'lugma', 'lugmuk', 'lugud', 'lugus', 'luka', 'lukas', 'lukib', 'lukluk', 'luklukan', 'lukman', 'luksu', 'luktuk', 'lukub', 'lukung', 'lukup', 'lukut', 'lula', 'lulam', 'lulan', 'lulub', 'lulud', 'lulun', 'lulut', 'lulwal', 'luma', 'lumakad', 'lumang', 'lumapit', 'lumawe', 'lumbat', 'lumboy', 'lumbu', 'lume', 'lumibut', 'lumipat', 'lumpia', 'lumwal', 'lungub', 'lunis', 'lunisantu', 'luntong', 'lupa', 'lupang', 'lupig', 'lupisak', 'lura', 'luslus', 'lusub', 'lusuk', 'lutang', 'lutas', 'lutlut', 'lutu', 'lutung', 'luyus', 'lwa', 'lwal', 'lwat', 'ma', 'maalikabuk', 'maam', 'mababa', 'mababalisa', 'mababating', 'mabagal', 'mabalacat', 'mabanglu', 'mabating', 'mabayat', 'mabengi', 'mabibyeng', 'mabilis', 'mabilug', 'mabubusit', 'mabulas', 'mabuluk', 'mabunduk', 'maburi', 'mabusni', 'mabye', 'macabebe', 'madagul', 'madalumdum', 'madinat', 'madisiplina', 'mag', 'magagamit', 'magalahas', 'magalang', 'magaling', 'maganaka', 'magaral', 'magbakasyun', 'magbalak', 'magbayu', 'magbisikleta', 'magbotu', 'magbus', 'magbyahe', 'magdala', 'magdalang', 'magdaragis', 'magdatun', 'magdilidili', 'magdisisyun', 'magdoktor', 'magexercise', 'magigisan', 'maging', 'maginu', 'magisan', 'magisplika', 'magkamali', 'magkanta', 'magkanu', 'magkanwaring', 'magkasakit', 'maglage', 'maglakad', 'maglakadlakad', 'maglalage', 'maglambat', 'maglaram', 'maglibe', 'maglima', 'maglutu', 'maglutung', 'magmaneho', 'magmaneu', 'magmaneung', 'magmovie', 'magnipun', 'magobra', 'magobrang', 'magorder', 'magpagana', 'magpaganaka', 'magpakilala', 'magpakimut', 'magpasikan', 'magpaynawa', 'magplanu', 'magrebyu', 'magrecycle', 'magreklamu', 'magrelax', 'magsadiang', 'magsadya', 'magsalita', 'magsalitang', 'magsane', 'magsaya', 'magscroll', 'magsilbi', 'magsingku', 'magsisi', 'magtagumpe', 'magtaka', 'magtaxi', 'magulu', 'magumpisa', 'magurung', 'magyelu', 'maka', 'makaba', 'makabaldugan', 'makabang', 'makabangmakaba', 'makabuklat', 'makabusni', 'makadwa', 'makagising', 'makaing', 'makaintindi', 'makaintinding', 'makakawa', 'makakilala', 'makalawe', 'makananu', 'makananung', 'makanyan', 'makapaglambat', 'makapagmaneu', 'makapagmulala', 'makapagparla', 'makapagpaynawa', 'makapagsalita', 'makapanaya', 'makapasno', 'makapinsalang', 'makapunta', 'makapuntang', 'makapunu', 'makarine', 'makasabi', 'makasadya', 'makasake', 'makasara', 'makasaup', 'makasukul', 'makatakas', 'makatakot', 'makatakut', 'makatatlu', 'makatikdo', 'makatua', 'makatudtud', 'makatuknang', 'makatula', 'makatulang', 'makatutwanan', 'maki', 'makikit', 'makilabas', 'makipagugnayan', 'makipagvideo', 'makiramdam', 'makisabi', 'makitalamitam', 'makiyagnan', 'maklak', 'makmak', 'mako', 'makulimlim', 'makuswelu', 'makuyad', 'makwalta', 'mal', 'mala', 'malaaldo', 'malabo', 'malabug', 'malagad', 'malagu', 'malakapas', 'malala', 'malalam', 'malamam', 'malamarine', 'malambat', 'malan', 'malangi', 'malapit', 'malasebu', 'malat', 'malati', 'malating', 'malatingmalati', 'malaus', 'malawut', 'malda', 'male', 'mali', 'malibagu', 'maligosu', 'maligwa', 'malikmata', 'malikwatas', 'malilibang', 'malilyari', 'malim', 'malinamnam', 'malingmingan', 'malinis', 'malino', 'malinong', 'maliputo', 'malisangan', 'malit', 'mall', 'malmal', 'malpa', 'malsinta', 'malulam', 'malun', 'malunggay', 'malungkut', 'malutak', 'malutu', 'malyare', 'malyari', 'malyaring', 'malyasa', 'mama', 'mamabayad', 'mamadwas', 'mamadyet', 'mamako', 'mamalagwa', 'mamali', 'mamaligwa', 'mamangan', 'mamasa', 'mamasyal', 'mamatras', 'mamayad', 'mamayun', 'mamialung', 'mamiye', 'mamulang', 'mamulut', 'mamunduk', 'mamyalung', 'mamye', 'man', 'mana', 'manakit', 'manalastas', 'manalbe', 'manalbeng', 'mananabu', 'mananam', 'mananu', 'manasan', 'manatbus', 'manatili', 'manatiling', 'manatu', 'manatul', 'manaya', 'manayang', 'mandatun', 'mandilu', 'manerbyus', 'mang', 'manga', 'mangalating', 'mangalilangan', 'mangamanu', 'mangan', 'mangang', 'mangaragul', 'mangasanting', 'mangaylangan', 'mangga', 'mangibili', 'mangisnawa', 'mangkuk', 'mangkukuran', 'mangmang', 'mangorteblan', 'mangotablan', 'mangulisak', 'mangutang', 'mangutatan', 'manibala', 'manibat', 'manigaral', 'manik', 'maninang', 'manintun', 'maniwala', 'manlapas', 'mansanas', 'manta', 'manuk', 'manuknangan', 'manuyang', 'manyad', 'manyali', 'manyaman', 'manyara', 'mapa', 'mapagal', 'mapali', 'mapaling', 'mapamangan', 'mapamasa', 'mapanganib', 'maparas', 'mapekat', 'mapilan', 'maputi', 'maragul', 'marakal', 'maralas', 'marangle', 'maranun', 'maranup', 'marapat', 'maraya', 'marayu', 'maria', 'marimla', 'marinat', 'marok', 'mars', 'martis', 'mary', 'maryng', 'mas', 'masabal', 'masakit', 'masala', 'masaleng', 'masalese', 'masaleseng', 'masalusu', 'masanting', 'masantol', 'masaplalang', 'masasakit', 'masaya', 'masayang', 'masid', 'masigla', 'masikan', 'masipag', 'masmas', 'masustansyang', 'maswerti', 'masyadu', 'masyadung', 'mata', 'matahimik', 'matalik', 'mataloti', 'mataluras', 'matalusad', 'matamad', 'matanggap', 'matapang', 'mataram', 'matas', 'mataung', 'matawli', 'mate', 'matenakan', 'math', 'matibe', 'matudtud', 'matudud', 'matula', 'matuling', 'matwa', 'matwang', 'maugtu', 'maulaga', 'maulagang', 'mauli', 'maulu', 'maumuk', 'mauran', 'maus', 'mawa', 'mayamaya', 'mayantoc', 'mayap', 'mayari', 'maybug', 'maygit', 'mayumu', 'mazapan', 'me', 'mebaril', 'mebating', 'mebigla', 'mebulag', 'mebyasa', 'media', 'medya', 'medyas', 'medyu', 'megbook', 'meging', 'megisan', 'meglutu', 'megobra', 'megpaganaka', 'megplanu', 'megshopping', 'megumpisang', 'mekabasa', 'mekaligtas', 'mekalingwan', 'mekamura', 'mekapagdisisyun', 'mekapamasa', 'mekapamayad', 'mekasulagpo', 'mekatadwa', 'mekatapilan', 'mekatudtud', 'mekauli', 'mekeni', 'meket', 'mekibat', 'mekigalame', 'mekikaluguran', 'mekiramdam', 'meko', 'melakwan', 'memakbung', 'mematda', 'memorya', 'memulayi', 'memyeng', 'menabu', 'menakit', 'menalbe', 'menalili', 'menasa', 'menasakit', 'menatili', 'menaya', 'meng', 'mengan', 'mengaylangan', 'menguku', 'mengutang', 'menibat', 'menik', 'menipun', 'mensahi', 'menu', 'menuknangan', 'menuru', 'menyad', 'menyangkap', 'mepata', 'meragul', 'merakap', 'mesakit', 'mesane', 'mesaya', 'mesingil', 'mesipit', 'mesira', 'mestra', 'mestro', 'mestru', 'mete', 'metung', 'mexico', 'meyaregla', 'meyari', 'mi', 'miabeabe', 'mibagsak', 'mibalik', 'mibalugse', 'miblas', 'mibulus', 'mibusni', 'mierkulis', 'migaganaka', 'migaganakaena', 'migdala', 'migdisisyun', 'migigising', 'migising', 'migkamali', 'migkulung', 'miglwal', 'migmaneu', 'migpalto', 'migpasya', 'migsalita', 'migumpisang', 'mika', 'mikakaluguran', 'mikakapatad', 'mikakasundu', 'mikakunu', 'mikalugud', 'mikaluguran', 'mikasundu', 'mikawani', 'mikit', 'mikitikit', 'mil', 'milabas', 'milabasan', 'milagrosa', 'milako', 'milalu', 'milalua', 'milayi', 'mililibe', 'milya', 'milyari', 'miminum', 'minagyu', 'minalin', 'minaliwa', 'ming', 'minindal', 'minom', 'minsan', 'minta', 'minuli', 'minum', 'minuran', 'minutu', 'minutus', 'mipaglibe', 'mipagnanu', 'mipagtennis', 'mipakadwa', 'mipakatlu', 'mipakibatan', 'mipamakibat', 'mipamanwa', 'mipamuklat', 'mipamulan', 'mipanuma', 'mipanuminggu', 'mipapangayli', 'mipapate', 'mipapatudtud', 'miparakal', 'miraras', 'miras', 'mirasan', 'miratang', 'misabi', 'misan', 'misane', 'misnang', 'mistula', 'mitakutan', 'mitatagan', 'mitauli', 'mitawli', 
'mitipun', 'mitla', 'mitmu', 'miya', 'miyabeabe', 'miyang', 'miyari', 'miyasawa', 'miyayaliwang', 'miyuranan', 'miyutput', 'mo', 'model', 'moncada', 'mong', 'moske', 'mosque', 'motor', 'motsi', 'mu', 'muko', 'muku', 'mukung', 'mukyat', 'mula', 'mulagat', 'mulala', 'mulang', 'mulat', 'muli', 'mulmul', 'mumuna', 
'mumunang', 'mumunta', 'mumuran', 'muna', 'munag', 'mune', 'mung', 'munta', 'mura', 'muran', 'muri', 'murin', 'musang', 'musangsang', 'musika', 'musing', 'muslo', 'musmus', 'mustasa', 'muta', 'mutya', 'muwang', 'mwa', 'na', 'nabang', 'nabengi', 'nabu', 'naga', 'naka', 'nakami', 'nakaming', 'nakang', 'nakatamu', 'nakeng', 'naknak', 'naku', 'nakung', 'nalang', 'naman', 'namang', 'name', 'namu', 'nana', 'nanamnaman', 'nanan', 'nananu', 'nang', 'nangnang', 'naning', 'nanita', 'nanu', 'nanung', 'nanupata', 'napnap', 'napun', 'nasa', 'nasi', 'nasnas', 'natto', 'nawa', 'nawang', 'nayun', 'ne', 'neman', 'neng', 'nengkayi', 'nerbyosung', 'nerbyus', 'new', 'ngabngab', 'ngalangala', 'ngalngal', 'ngan', 'ngana', 'ngang', 'nganga', 'ngangas', 'nganib', 'ngara', 'ngatba', 'ngatngat', 'ngene', 'ngeni', 'ngening', 'ngingil', 'ngisngis', 'ngongu', 'ngotngot', 'ngudngud', 'ngulangul', 'ngusngus', 'ngutal', 'ngutngut', 'ni', 'niknik', 'nin', 'ninanu', 'ning', 'ningnang', 'ningning', 'ninu', 'ninung', 'nita', 'niti', 'nku', 'no', 'nokarin', 'nokaring', 'nong', 'notes', 'nu', 'nuan', 'nuebi', 'nugut', 'nukarin', 'numanu', 'numeru', 'nune', 'nung', 'nunu', 'nyebe', 'o', 'obat', 'obispo', 'obliga', 'obligasion', 'obra', 'obrang', 'obrero', 'oh', 'oido', 'oini', 'okasi', 'okasyun', 'okoy', 'okra', 'oktubri', 'okupadu', 'oneng', 'online', 'onsi', 'oorderan', 'optiku', 'opu', 'orador', 'oras', 'order', 'ordin', 'oren', 'oreni', 'oreta', 'oreti', 'oriana', 'ortograpiya', 'orus', 'osigi', 'ospital', 'ostia', 'osyo', 'osyu', 'ot', 'otap', 'otsu', 'oven', 'oya', 'oyni', 'oyta', 'pa', 'pabalwan', 'pabanwa', 'pablasa', 'paborito', 'paboritu', 'pabrika', 'pabulan', 'paburen', 'pabureng', 'paburitu', 'pabustan', 'pacifico', 'pader', 'padwas', 'pagal', 'paganaka', 'pagaralan', 'pagawa', 'paginakit', 'pagkalangan', 'pagkeran', 'pagkit', 'paglalawe', 'pagmananwan', 'pagmaragul', 'pagmasusian', 'pagmayumu', 'pago', 'pagpag', 'pagprosesu', 'pagselebran', 'pagtuan', 'pagulung', 'paintulut', 'paintulutan', 'pakaban', 'pakabanten', 'pakabayu', 'pakabistan', 'pakabug', 'pakakak', 'pakakalale', 'pakalon', 'pakalulu', 'pakamal', 'pakamat', 'pakang', 'pakde', 'pake', 'pakibat', 'pakibatan', 
'pakidwag', 'pakiisplika', 'pakikitan', 'pakilako', 'pakilala', 'pakiplantsa', 'pakiramdam', 'pakiramdaman', 'pakisabi', 'pakisabian', 'pakisabyan', 'pakisalangyan', 'pakisara', 'pakit', 'pakiyabayan', 'paklud', 'pakpak', 'paksa', 'paksi', 'paku', 'pakyalam', 'pakyo', 'pala', 'palad', 'palage', 'palak', 'palako', 'palakul', 'palanakan', 'palang', 'palapa', 'palapag', 'palapat', 'palara', 'palaspas', 'palayo', 'paldak', 'paldakan', 'paldan', 'paldanuman', 'paldas', 'pale', 'palekpek', 'palengki', 'pali', 'palid', 'paligi', 'palimbun', 'paling', 'palino', 'palipi', 'palipit', 'palis', 'palitada', 'palitu', 'palmera', 'palpak', 'palpal', 'palpi', 'paltak', 'paltik', 'paltus', 'palu', 'palual', 'paluka', 'palunag', 'palune', 'palung', 'palungmanuk', 'palunpun', 'palupalu', 
'palus', 'palut', 'palwal', 'pamag', 'pamagasalita', 'pamagbayu', 'pamagbisikleta', 'pamagisplika', 'pamagkomyut', 'pamagkumbaba', 'pamaglabag', 'pamaglakad', 'pamaglakbe', 'pamaglalang', 'pamaglingap', 'pamagmaneho', 'pamagobra', 'pamagsulit', 'pamagsumikap', 'pamalak', 'pamalino', 'pamamangan', 'pamamupul', 'pamanagkat', 'pamananaman', 'pamanatu', 'pamangamanu', 'pamangamanung', 'pamangan', 'pamangaylangan', 'pamanintun', 'pamanyaup', 'pamanyulagpo', 'pamati', 'pambuk', 'pamepe', 'pamikakatan', 'pamikatagun', 'pamilya', 'pamintu', 'pamipakilala', 'pamisapungan', 'pamisulatmap', 'pamita', 'pamitipun', 'pamiyabe', 'pampam', 'pampano', 'pamu', 'pamuga', 'pamugsuk', 'pamun', 'pamung', 'pana', 'panagkas', 'panakitan', 'panakitana', 'panalangin', 'panamdaman', 'panara', 'panas', 'panaun', 'panaung', 'panaya', 'panayan', 'pancit', 'pandak', 'pandakaki', 'pandan', 'pandasal', 'pande', 'pandig', 'pandung', 'pane', 'panenayan', 'pang', 'panga', 'pangabalanan', 'pangadi', 'pangadwa', 'pangadwang', 'pangahudyu', 'pangaku', 'pangamestra', 'pangamestru', 'panganam', 'pangane', 'panganib', 'pangapat', 'pangasukul', 'pangat', 'pangatalaturu', 'pangatlu', 'pangawala', 'pangilagan', 'pangiling', 'pangkat', 'panglima', 'pangpang', 'pangsul', 'pangulu', 'pangunakan', 'panibalan', 'panig', 'panigapu', 'panigaralan', 'panik', 'paniki', 'panilan', 'paninapan', 'paninintunan', 'panintun', 'panintunan', 
'paniqui', 'paniti', 'paniwalan', 'panotsa', 'panpan', 'pansamantala', 'pansit', 'pantalun', 'pantas', 'pantasya', 'pante', 'pantug', 'panugali', 'panuktukan', 'panulu', 'panupaya', 'panyabyan', 'panyalita', 'panyu', 'papag', 'papak', 'papalwan', 'papasyalan', 'papel', 'papunta', 'para', 'parak', 'paralan', 'paralaya', 'param', 'paraman', 'paramanang', 'parang', 'parangparang', 'paraparuminggu', 'paras', 'parati', 'pare', 'pareha', 'parehu', 'pareu', 'pareung', 'parewu', 'pari', 'parilya', 'paris', 'parisukad', 'paritan', 'park', 'paro', 'paroba', 'parokya', 'parte', 'party', 'parul', 'paruleru', 'paruminggu', 'paruminggung', 'parus', 'parusan', 'parusparus', 'pasa', 'pasaherus', 'pasake', 'pasalamat', 'pasalubung', 'pasalungat', 'pasamantala', 'pasamanu', 'pasaporti', 
'pasas', 'pasayan', 'pasbul', 'pasdan', 'pase', 'pasensya', 'pasi', 'pasibayo', 'pasibayu', 'pasig', 'pasimuna', 'pasimundut', 'pasitalan', 'pasku', 'paskwa', 'pasla', 'paslu', 'pasma', 'paspas', 'pasta', 'pastilan', 'pastul', 'pasu', 'pasubali', 'pasunud', 'paswang', 'paswit', 'pasya', 'pasyag', 'pasyal', 'pasyosyo', 'pasyuk', 'pasyun', 'pat', 'pata', 'pataba', 'patad', 'patag', 'patak', 'patakaran', 'patalan', 'patan', 'patas', 'patatas', 'patawan', 'patda', 'patdan', 'pate', 'patepate', 'pati', 'patianak', 'patid', 'pating', 'patingapun', 'patipati', 'patis', 'patiyanak', 'patlalu', 'patlud', 'patnubayan', 'patnube', 'patnugut', 'patola', 'patse', 'patugut', 'patulunan', 'patungkul', 'patupat', 'pau', 'paugtwan', 'pauli', 'pawikan', 'payapa', 'payarap', 'payat', 'payaus', 'payid', 'payidpid', 'payit', 'payntunan', 'payntungulan', 'payo', 'payu', 'payul', 'payung', 'payungpayungan', 'payus', 'pe', 'peburen', 'peka', 'pekaburi', 'pekamalagung', 'pekamalapad', 'pekamalapit', 'pekamaragul', 'pekamarok', 'pekamasanting', 'pekamatas', 'pekisabyan', 'pelangga', 'pelikula', 'pelikulang', 'penako', 'penaya', 'pengari', 'peniwalan', 'penyali', 'penyaling', 'pepabalu', 'pepabanwa', 'pepabulan', 'pepabye', 'pepagaral', 'pepagawa', 'pepakit', 'pepalto', 'pepamunikala', 'pepapintura', 'pepapuntan', 'peparagul', 'pepasaup', 'pepatingapun', 'pepaula', 'pepawaga', 'pepayaus', 'pepe', 'pepepe', 'pera', 'perang', 'pero', 'perpektu', 'pesa', 'pete', 'pianu', 'pibandian', 'piesta', 'piga', 'pigil', 'piknik', 'pikpik', 'pila', 'pilak', 'pilan', 'pilang', 'pilapil', 'pilatan', 'pildis', 'pile', 'pili', 'pilidu', 'piling', 'pilit', 'pilubluban', 'pin', 'pinak', 'pinan', 'pinandit', 'pindang', 'pinduan', 'ping', 'pingas', 'pinggan', 'pingul', 'pinilit', 'pinocha', 'pinpin', 'pinsan', 'pinta', 'pintakasi', 'pintas', 'pintor', 'pintu', 'pipagaralan', 'pipanganan', 'pipi', 'pipiling', 'pipinu', 'pipis', 'pipitsugin', 'pirasu', 'pirayit', 'piring', 'piris', 'piruya', 'pisabyan', 'pisak', 'pisalwanan', 'pisan', 'pisasabian', 'pisi', 'pisig', 'pisik', 'pisni', 'pispis', 'pistu', 'pitak', 'pitaka', 'pitaklan', 'pitchipitchi', 'pitda', 'pitik', 'pitityan', 'pitna', 'pitpit', 
'pitse', 'pitsu', 'pitu', 'pituklip', 'pitupitu', 'piunyaban', 'piyun', 'pizza', 'plaisdan', 'plaka', 'plakda', 'plake', 'plan', 'planta', 'planu', 'plasta', 'platitu', 'plato', 'poema', 'poesiya', 'poeta', 'polusyun', 'porac', 'pormal', 'posibli', 'pospas', 'poste', 'postre', 'posu', 'pota', 'potang', 'potru', 'pranses', 'premyu', 'preparadu', 'presku', 'presu', 'presyu', 'prisensya', 'pritil', 'pritung', 'problema', 'propesyunal', 'prosesu', 'prublema', 'prublemang', 'prutas', 'psychic', 'pu', 'puapu', 'puga', 'puge', 'pugit', 'pugita', 'pugu', 'pugung', 'pukat', 'pukendang', 'puki', 'pukpuk', 'puksa', 'pukul', 'pukyutan', 'pulandit', 'pulayi', 'pulgas', 'puli', 'puling', 'pulis', 'pulosa', 'pulu', 'pulung', 'pulunggus', 'pulupulu', 'pulus', 'pulut', 'pulutuk', 'pun', 'puna', 'punda', 'pundasyun', 'pung', 'pungapung', 'pungguk', 'pungkul', 'pungpung', 'pungsu', 'pungul', 'pungus', 'puni', 'punla', 'punta', 'puntalan', 'puntas', 'puntetas', 'punti', 'puntos', 'puntu', 'puntuk', 'puntus', 'punyus', 'pupud', 'pupul', 'pupunta', 'pupuntalan', 'pupus', 'puput', 'pura', 'purak', 
'purga', 'puri', 'puru', 'purud', 'puruk', 'purul', 'purus', 'pusa', 'pusad', 'pusan', 'pusang', 'pusit', 'pusitara', 'pusu', 'pusuk', 'pusung', 'puta', 'putahi', 'putakti', 'putat', 'puti', 'putla', 'putput', 'putris', 'putu', 'putung', 'putuseku', 'putut', 'puyat', 'pwad', 'pweding', 'pyalung', 'pyalungan', 'pyanu', 'quebiawan', 'quenabuan', 'quilayin', 'quilo', 'quisa', 'ra', 'radyo', 'radyu', 'raha', 'raka', 'rakang', 'rakayu', 'raketa', 'rambutan', 'ramen', 'ramos', 'rampa', 'rang', 'ranku', 'rantsu', 'raspa', 'rayus', 're', 'reas', 'rebelde', 'rebentador', 'rebosadu', 'rebultu', 'rebweltu', 'recipe', 'redentor', 'reforma', 'refund', 'regalu', 'regla', 'rehas', 'reina', 'rekadu', 'reklamu', 'rekwerdu', 'relihion', 'relikia', 'reliyebo', 'relleno', 'relo', 'remata', 'rematsi', 'remedyu', 'ren', 'renda', 'reng', 'renta', 'repike', 'repinadu', 'repolyu', 'report', 'resaresa', 'reseta', 'resibu', 'residenti', 'resolban', 
'respetu', 'respetwan', 'responsabli', 'restoran', 'reta', 'retasu', 'retiru', 'retoki', 'retratu', 'reunion', 'reyuma', 'ribelde', 'rigalu', 'rilo', 'rima', 'rimadu', 'rimas', 'rimedyu', 'rin', 'ring', 'ritasu', 'role', 'ronda', 'rondalya', 'rosal', 'rosario', 'rosas', 'rueda', 'rugu', 'runut', 'ruta', 'rutina', 'rweda', 'sa', 'saba', 'sabadu', 'sabaduduminggu', 'sabadung', 'sabagal', 'sabagkal', 'sabak', 'sabalbal', 'sabalo', 'sabana', 'sabat', 'sabatan', 'sabe', 'sabesabe', 'sabi', 'sabian', 'sabik', 'sabila', 'sabit', 'sabla', 'sable', 'sabo', 'sabpang', 'sabsab', 'sabu', 'sabug', 'sabul', 'sabung', 'sabwat', 'sabyan', 'sabyang', 'sadsad', 'sadya', 'sadyang', 'safety', 'sagad', 'sagakgak', 'sagana', 'saganan', 'sagap', 'sagasa', 'sagese', 'sagid', 'sagimgim', 'sagin', 'sagip', 'sagipan', 'sagisag', 'sagiwa', 'sagiwang', 'sagka', 'sagkil', 'sagkul', 'saglap', 'saglawe', 'saglit', 'sagmit', 'sagoso', 'sagsag', 'sagu', 'sagudgud', 'saguli', 'sagupa', 'saguyud', 'sagwake', 'saingsing', 'sais', 'saka', 'sakab', 'sakali', 'sakang', 'sakate', 'sakbali', 'sakbibi', 'sakbit', 'sake', 'saken', 'sakildap', 'sakim', 'sakit', 'sakle', 'saklit', 'saklo', 'saklu', 'saklulu', 'saklung', 'sakmal', 'sakobi', 'sakoru', 'sakoti', 'saksak', 'saksi', 'sakul', 'sakuna', 'sakung', 'sakup', 'sakyud', 'sakyuran', 'sala', 'salab', 'salabat', 'salabsab', 'salak', 'salakab', 'salakata', 'salake', 'salakup', 'salamat', 'salamin', 'salampak', 'salamuha', 'salandra', 'salang', 'salangat', 'salangi', 'salangsalang', 'salangsang', 'salansan', 'salap', 'salapi', 'sale', 'sali', 'salilungan', 'saling', 'salita', 'saluduran', 'salusad', 'salwan', 'samal', 'samantalanan', 'samaral', 'samatra', 'sambra', 'sampaga', 'sampagang', 'sampaluk', 'sampelut', 'sana', 'sang', 'sanga', 'sanggayid', 'sangkap', 'santan', 'santing', 'santul', 'sapak', 'sapatus', 'sapoti', 'sapsap', 'sapsoy', 'sapwak', 'sara', 'saramulyeti', 'sarili', 'sarulgamat', 'sasabi', 'sasabian', 'sasabyan', 'sasag', 'sasake', 'sasalikut', 'sasamasan', 'sasambut', 'sasangkanan', 'sasaryan', 'sasaup', 'sasmuan', 'saup', 'saupan', 'saya', 'sayang', 'sayoti', 'sayu', 'school', 'seafood', 'seatbelt', 'sebulan', 'seda', 'segundo', 'seis', 'sekretaryang', 'selan', 'selda', 'seli', 'selikut', 'selpon', 'selyu', 'semana', 'semilya', 'sensi', 'sentimetrung', 'senyas', 'sepu', 'seryusung', 'sese', 'sesenta', 'setenta', 'sex', 'shabu', 'shawarma', 'siak', 'siam', 'siansi', 'sias', 'siasat', 'sibad', 'sibangut', 'sibil', 'sibli', 'sibug', 'sibukan', 'sibul', 'sibut', 'sibuyas', 'siempre', 'siensi', 'sientu', 'sieti', 'siga', 'sigalut', 'sigang', 'sigasig', 'sige', 'sigi', 'sigla', 'sigma', 'signus', 'sigpo', 'siguradu', 'siguraduan', 'siguru', 'sigurung', 'sigwa', 'sikad', 'sikamas', 'sikan', 'sikanan', 'sikang', 'sikap', 'sikasuan', 'sikat', 'sikawat', 'sikit', 'siklab', 'siklod', 'sikluban', 'sikretu', 'sikretung', 'siku', 'sikuaku', 'sikutsikut', 'sila', 'silab', 'silaba', 'silad', 'silais', 'silbatu', 'silbi', 'silid', 'silim', 'silimsilim', 'silindru', 'silinsilinan', 'silinyasi', 'silio', 'silip', 'silo', 'silu', 'silung', 'silya', 'silyo', 'sim', 'simarun', 'simba', 'simboryu', 'simbung', 'simbut', 'sime', 'simpan', 'simpatya', 'simpleng', 'simpli', 'simulmul', 'simut', 'sinabi', 'sinali', 'sinamay', 'sinambut', 'sinantan', 'sinawa', 'sinayad', 'sindak', 'sindi', 'sindu', 'sine', 'sinelas', 'singap', 'singil', 'singit', 'singkamas', 'singku', 'singkul', 'singkulima', 'singkusingku', 'singkwenta', 'singut', 'sinilip', 'sining', 'sinopan', 'sinsil', 'sinsin', 'sinta', 'sintak', 'sintas', 'sintidu', 'sintunadu', 'sinturun', 'sinubuk', 'sinuku', 'sinulagpo', 'sinulapo', 'sinulat', 'sinunga', 'sinup', 'siopao', 'sipag', 'sipak', 'siping', 'sipit', 'sipsip', 'sipun', 'sir', 'sira', 'siri', 'siryosu', 'siryosuan', 'sisi', 'sisig', 'sisiguraduan', 'sisikapan', 'sisilim', 'sisilyu', 'sisindi', 'sisiran', 'sisti', 'sisyan', 'sitsarun', 'sitsirika', 'sitsirya', 'sitsit', 'sitwasyun', 'siwala', 'siwang', 'skyscraper', 'sobra', 'sobrang', 'sobri', 'soccer', 'social', 'soft', 'solteru', 'solu', 'soning', 'sopan', 'sopas', 'sopla', 'soso', 'sosopan', 'soswa', 'sota', 'sotanghun', 'spanish', 'stock', 'stop', 'street', 'stress', 'stressful', 'sua', 'suba', 'subali', 'subasta', 'subidu', 'subli', 'subsub', 'subu', 'subuk', 'subukan', 'subung', 'sudsud', 'sugal', 'sugapa', 'sugarul', 'sugat', 'sugi', 'sugid', 'suglung', 'sugne', 'sugpu', 'sugu', 'sugud', 'suka', 'sukad', 'sukal', 'sukat', 'sukdul', 'suki', 'sukib', 'suklab', 'suklam', 'suklati', 'sukle', 'sukli', 'suklub', 'suksuk', 'suku', 'sukulan', 'sulagpo', 'sulak', 'sulaksulak', 'sular', 'sulat', 'sulatanan', 'sulbatana', 'suldut', 'suli', 'sulib', 'sulparit', 'sulpit', 'sulput', 'sulsi', 'sulsul', 'sultan', 'sulu', 'sulud', 'sulugi', 'suluk', 'sulung', 'sulusulu', 'sulut', 'sulyap', 'suma', 'sumambut', 'sumami', 'suman', 'sumangid', 'sumanting', 'sumaup', 'sumbagal', 'sumbat', 'sumbung', 'summer', 'sumpa', 'sumpal', 'sumpung', 'sumuku', 'sumulung', 'sundalus', 'sundang', 'sungit', 'sungka', 'sungki', 'sungkit', 'sunis', 'suntuk', 'supil', 'supladu', 'suporta', 'suportang', 'suput', 'surbatu', 
'sushi', 'susi', 'sustansiya', 'sustentu', 'susu', 'susubukan', 'susulat', 'susut', 'susyal', 'sut', 'sutil', 'sutla', 'suyu', 'swa', 'swang', 'swekus', 'sweldu', 'swi', 'syam', 'syang', 'syete', 'syeti', 'syudad', 'ta', 'taba', 'tabak', 'tabaku', 'tabalu', 'tabang', 'tabas', 'tabi', 'tabili', 'tabing', 'tabla', 'tablan', 'tableta', 'tabon', 'tabrete', 'tabsing', 'tabtab', 'tabu', 'tabug', 'tabun', 'tabureti', 'tadtad', 'taga', 'tagabili', 'tagak', 'tagal', 'tagalug', 'tagan', 'tagana', 'taganang', 'tagibang', 'tagilid', 'tagilo', 'tagimpan', 'tagin', 'tagis', 'tagkil', 'tagle', 'taglus', 'tagnan', 'tagpi', 'tagpus', 'tagtag', 'tagu', 'tagulale', 'tagulele', 'tagun', 'taguyud', 'tahimik', 'tahu', 'taid', 'taimik', 'tais', 'takam', 'takap', 'takas', 'takba', 'takbang', 'takde', 'take', 'takid', 'takla', 'taklaid', 'takman', 'taknaido', 'taksantaksan', 'taksiapo', 'taksil', 'taktak', 'taku', 'takub', 'takuku', 'takung', 'takuri', 'takusa', 'takut', 'tal', 'tala', 'talab', 'talaba', 'talabaldugan', 'talabu', 'taladtad', 'talaga', 'talaib', 'talakad', 'talakituk', 'talaksan', 'talamitam', 'talampakan', 'talamsik', 'talan', 'talanan', 'talandi', 'talang', 'talanga', 'talangka', 'talangkibi', 'talangtalang', 'talangtang', 'talasawa', 
'talastas', 'talasulat', 'talatag', 'talatuki', 'talaturu', 'talausuk', 'talbug', 'talbus', 'talete', 'tali', 'taliasi', 'talibatab', 'talik', 'talikala', 'talikbung', 'talilis', 'talimhaga', 'talimpukan', 'talimunduk', 'talimusak', 'talindata', 'talindiking', 'talinu', 'talip', 'talipapa', 'taliri', 'talisud', 'talukab', 'taluki', 'taluktuk', 'talum', 'talumparit', 'talumpati', 'talunasan', 'talung', 'talungku', 'talyasi', 'tamad', 'tamales', 'tamalis', 'tamasa', 'tambak', 'tamban', 'tambayuk', 'tambing', 'tambor', 'tambubung', 'tambul', 'tambuli', 'tambun', 'tamimi', 'tamle', 'tampa', 'tampal', 'tampalasan', 'tampaling', 'tampan', 'tampiso', 'tampu', 'tampuk', 'tamu', 'tamumu', 'tamung', 'tana', 'tanak', 'tanam', 'tanaman', 'tanda', 'tandang', 'tandayan', 'tando', 'tandus', 'tang', 'tanga', 'tangab', 'tanggal', 'tanggap', 'tanggi', 'tanggilu', 'tanghun', 'tangka', 'tangke', 'tangki', 'tangkilik', 'tanglad', 'tangle', 'tangnan', 'tangsu', 'tangtang', 'tangu', 'tanikala', 'taning', 'tansan', 'tantu', 'tantya', 'tanud', 'tanyag', 'tapa', 'tapak', 'tapal', 'tapalodu', 'tapang', 'tapangku', 'tapas', 'tapat', 'tapayan', 'tapeti', 'tapi', 'tapik', 'tapioka', 'tapis', 'tapitapi', 'tapius', 'tapuk', 'tapun', 'tapung', 'tapyas', 'tara', 'tarak', 'taram', 'taramindu', 'taranati', 'tarangka', 'taranta', 'tarantadu', 'tarantela', 'tarapal', 'taras', 'tarat', 'tarebalak', 'tariji', 'tarik', 'taripa', 'tarlac', 'tarlak', 'tarling', 'tartanilya', 'taru', 'taruk', 'tas', 'tasa', 'tasak', 'tasik', 'tastas', 'tata', 'tatabili', 'tatad', 'tatag', 'tatak', 'tatakut', 'tatal', 'tatalangan', 'tatalnan', 'tatang', 'tatanggap', 'tatsa', 'tatsu', 'tatu', 'tatyo', 'tau', 'tauge', 'tauli', 'tauling', 'taung', 'tausi', 'taute', 'tauwan', 'tawad', 'tawas', 'tawilis', 'tawli', 'taya', 'tayag', 'tayang', 'tayi', 'taysan', 'tayum', 'te', 'tegawan', 'tela', 'telabanua', 'telatag', 'teleponu', 'telon', 'tema', 'tembong', 'tennis', 'tepangan', 'terak', 'terno', 'tesoro', 'testamentu', 'testigu', 'testimonya', 'tete', 'ti', 'tia', 'tiak', 'tiba', 'tibabal', 'tibag', 'tibak', 'tiban', 'tibe', 'tibis', 'tibuk', 'tibuktibuk', 'tidtad', 'tidwasingku', 'tigab', 'tigagal', 'tigang', 'tigit', 'tigpas', 'tigre', 'tigsa', 'tigtig', 'tika', 'tikbalang', 'tikdo', 'tiket', 'tikyan', 'tilak', 'tilapia', 'tilauk', 'tilis', 'tilsuk', 'tiltil', 'tiltilan', 'tilus', 'timan', 'timawa', 'timba', 'timbang', 'timbre', 'time', 'timid', 'timog', 'timpak', 'timpapalis', 'timpla', 'timpu', 'timpukan', 'tin', 'tina', 'tinagun', 'tinakas', 'tinakbang', 'tinaklub', 'tinapa', 'tinape', 'tinawak', 'tinda', 'tindag', 'tindahan', 'tindera', 'tinext', 'tingga', 'tinggap', 'tingkad', 'tingting', 'tinikdo', 'tinikul', 'tiniman', 'tinola', 'tinuki', 'tinuknang', 'tinukup', 'tinulak', 'tipa', 'tipak', 'tipan', 'tipid', 'tiplad', 'tipun', 'tira', 'tirada', 'tirador', 'tiran', 'tirante', 'tirintas', 'tirintin', 'tiris', 'tiru', 'tisa', 'titi', 'titigtig', 'titiman', 'titlu', 'titlusingku', 'tiu', 'tiwala', 'tiwalag', 'tiyaga', 'tiyani', 'todas', 'todu', 'todutodu', 'tofu', 'toga', 'toge', 'toka', 'tokayo', 'tokwa', 'tolda', 'tom', 'tomba', 'tombalevanta', 'tompyang', 'tonto', 'tonu', 'topograpya', 'topotopo', 'torero', 'tori', 'tormentu', 'torneo', 'tornilyo', 'torno', 'toro', 'torotot', 'torpe', 'torso', 'torta', 'tortilia', 'tosu', 'toto', 'toyu', 'tradisiyun', 'traffic', 'trahe', 'trahedya', 'traidul', 'trak', 'traktora', 'trambia', 'tramu', 'trangkasu', 'transisyon', 'transportasyun', 'trapik', 'tratadu', 'tratu', 'traviesa', 'treinta', 'tren', 'trenta', 'tres', 'tresi', 'tresyientus', 'trianggulu', 'tribu', 'tribuna', 'tribunal', 'tricycle', 'trigo', 'triple', 'trofeo', 'trombon', 'trompa', 'trompeta', 'tronu', 'tropa', 'trumpu', 'tsa', 'tsaka', 'tsalima', 'tsamba', 'tsampuradu', 'tsanggi', 'tsansa', 'tsika', 'tsiklet', 'tsiku', 'tsitsirika', 'tsokolati', 'tsunami', 'tua', 'tuag', 'tualya', 'tuas', 'tuastuasan', 'tuberu', 'tubo', 'tubu', 'tubud', 'tud', 'tudtud', 'tuerka', 'tugak', 'tugis', 'tuglung', 'tugma', 'tugtug', 'tugtugan', 'tugun', 'tugut', 'tukador', 'tukayu', 'tuki', 'tukil', 'tukituki', 'tuklap', 'tuklas', 'tuklip', 'tuklu', 'tuknang', 'tuknangan', 'tuksu', 'tuktuk', 'tuku', 'tukud', 'tukudbanua', 'tukung', 'tukup', 'tukyan', 'tula', 'tulad', 'tulak', 'tulala', 'tulang', 'tulari', 'tulauk', 'tuldu', 'tuli', 'tulia', 'tuliasi', 'tulid', 'tuling', 'tulisan', 'tulsik', 'tulu', 'tulud', 'tuluk', 'tulus', 'tulut', 'tulya', 'tumagun', 'tumakas', 'tumbas', 'tumbuk', 'tumbung', 'tumerak', 'tumiman', 'tumor', 'tumpak', 'tumpuk', 'tumpyak', 'tun', 'tuna', 'tundun', 'tune', 'tunganga', 'tunggak', 'tunggal', 'tunggali', 'tungge', 'tungi', 'tungku', 'tungkul', 'tungkulan', 'tungkus', 'tungo', 'tungtung', 'tunsuy', 'tuntun', 'tunud', 'tupa', 'tupad', 'tuparan', 'turing', 'turkesa', 'turkia', 'turko', 'turu', 'turun', 'turung', 'tustus', 'tusuk', 'tuta', 'tuto', 'tutu', 'tutubu', 'tutuk', 'tutuki', 'tutuking', 'tutul', 'tutung', 'tutup', 'tuya', 'tuyu', 'tv', 'twitter', 'uamd', 'ubas', 'ubasubas', 'ubat', 'ubi', 'ubingan', 'ubu', 'ubud', 'ubuk', 'ugtu', 'uguyuguy', 'ukdu', 'ukuk', 'ukul', 'ukum', 'ukyat', 'ulabe', 'ulad', 
'ulan', 'ulang', 'ulap', 'ulas', 'ulat', 'ulata', 'ulbuk', 'uli', 'ulian', 'ulikba', 'ulila', 'ulimo', 'uling', 'ulip', 'ulit', 'ulma', 'ulsera', 'ultimu', 'ulu', 'ulun', 'ulus', 'uma', 'uman', 'ume', 'umis', 'umpisa', 'umpisan', 'una', 'unanu', 'undap', 'unduk', 'une', 'ungkat', 'ungul', 'ungut', 'unibersidad', 'uniberso', 'unico', 'unipormi', 'unlad', 'unu', 'unus', 'unyab', 'unyat', 'upaya', 'upisina', 'uran', 'uri', 'uria', 'urna', 'urud', 'urung', 'urungsulung', 'urut', 'usa', 'usbung', 'usig', 'using', 'usisa', 'usisero', 'usta', 'ustu', 'ustung', 'usu', 'usuk', 'usup', 'usura', 'usureru', 'utak', 'utal', 'utang', 'utap', 'utik', 'util', 'utu', 'utul', 'utus', 'utut', 'uwa', 'uwanan', 'uwang', 'uwawa', 'uwis', 'uy', 'uyab', 'valdez', 'valido', 'valiente', 'vampira', 'vandalismo', 'vanguardia', 'vanidad', 'vanidoso', 'vanilya', 'variasyon', 'vasyo', 'veinte', 'velada', 'velo', 'venerable', 'vengador', 'venganza', 'vengativo', 'venta', 'ventahe', 'ventanilya', 'ventilador', 'ventosa', 'verbal', 'verbo', 'verdad', 'verdadero', 'verdugo', 'verguenza', 'versikulo', 'verso', 
'vertigo', 'vespera', 'veterano', 'veterinario', 'vez', 'via', 'viakrusis', 'vianda', 'victor', 'victoria', 'vida', 'viernis', 'vietnam', 'villa', 'violeta', 'violin', 'virgen', 'virtud', 'visita', 'visitas', 'visitasyon', 'visperas', 'viva', 'vokabulario', 'vokadura', 'vokal', 'vokalista', 'vokasyon', 'volt', 'voluntaryu', 'vuelo', 'vuelta', 'vulcan', 'vulgarismu', 'vulnerabli', 'wa', 'waga', 'wagas', 'wagwag', 'wak', 'wakas', 'wakat', 'wake', 'waksi', 'wakwak', 'wala', 'walak', 'wale', 'wali', 'walingwaling', 'walu', 'wanan', 'wangis', 'wansuy', 'wapen', 'waray', 'wardi', 'wari', 'wasak', 'wasiwas', 'watas', 'watawat', 'wato', 'wawa', 'webis', 'weekend', 'wili', 'wis', 'wok', 'xenia', 'xenon', 'xerox', 'xmas', 'xray', 'xylophone', 'y', 'ya', 'yabang', 'yabut', 'yabyab', 'yagyag', 'yaku', 'yakyak', 'yalu', 'yalung', 'yama', 'yaman', 'yampa', 'yamu', 'yamuk', 'yamyam', 'yan', 'yana', 'yanasa', 'yang', 'yanga', 'yangka', 'yangyang', 'yantik', 'yantuk', 'yapa', 'yapyap', 'yarda', 'yari', 'yata', 'yatad', 'yatiat', 'yatu', 'yawad', 'yaya', 'yayat', 'yayta', 'ye', 'yelu', 'yeru', 'york', 'yu', 'yubyub', 'yugyug', 'yuku', 'yukung', 'yumu', 'yung', 'yungyung', 'yupi', 'yutuk', 'yuyu', 'yuyut', 'zambales', 'zapote', 'zaragoza', 'zarzaparilla', 'zarzuela', 'zero', 'zeta', 'zionismu', 'zona']     # English (Latin script)

In [ ]:
import pandas as pd

df = pd.read_csv(CSV_PATH)
print(df.columns)


# === 2. Load CSV ===

In [ ]:
df = pd.read_csv(CSV_PATH)
df = df.rename(columns={"kapampangan": "src_text", "english": "tgt_text"})
df = df.dropna(subset=["src_text", "tgt_text"])
df.head()

# === 3. Convert to HF Dataset ===

In [ ]:
dataset = Dataset.from_pandas(df[["src_text", "tgt_text"]])
dataset = dataset.train_test_split(test_size=0.2, seed=42)

import random
import nltk
from nltk.corpus import wordnet

nltk.download("wordnet")
nltk.download("omw-1.4")

def synonym_replacement_en(text, prob=0.05):
    """Replace English words with synonyms (low probability)."""
    words = text.split()
    new_words = []
    for w in words:
        if random.random() < prob:
            syns = wordnet.synsets(w)
            if syns:
                lemmas = syns[0].lemma_names()
                if lemmas:
                    w = random.choice(lemmas)
        new_words.append(w)
    return " ".join(new_words)

def noise_injection_kap(text, prob=0.01):
    """Simulate ASR spelling noise by swapping/deleting/inserting chars."""
    chars = list(text)
    for i in range(len(chars)):
        if random.random() < prob:
            op = random.choice(["swap","delete","insert"])
            if op == "swap" and i < len(chars)-1:
                chars[i], chars[i+1] = chars[i+1], chars[i]
            elif op == "delete":
                chars[i] = ""
            elif op == "insert":
                chars[i] = chars[i] + random.choice("aeiou")
    return "".join(chars)

def word_dropout(text, prob=0.02):
    """Randomly drop words to mimic ASR omissions."""
    words = text.split()
    return " ".join([w for w in words if random.random() > prob])

def augment_pair(src, tgt):
    # Source-side: ASR-like errors
    src = noise_injection_kap(src, prob=0.01)
    src = word_dropout(src, prob=0.02)
    
    # Target-side: synonym variation
    tgt = synonym_replacement_en(tgt, prob=0.05)
    
    return src, tgt

# === Fix 1: Augmentation function ===
def augment_dataset(ds, multiplier=2):
    new_data = {"src_text": [], "tgt_text": []}
    for example in ds:
        for _ in range(multiplier):
            src, tgt = augment_pair(example["src_text"], example["tgt_text"])
            new_data["src_text"].append(src)
            new_data["tgt_text"].append(tgt)
    # Convert back to HF dataset and concatenate
    aug_ds = Dataset.from_dict(new_data)
    return ds.concatenate(aug_ds)

# Only augment training set
dataset["train"] = augment_dataset(dataset["train"], multiplier=2)

# === 4. Load Tokenizer & Model ===

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Add special tokens
tokenizer.add_special_tokens({'additional_special_tokens': [SPECIAL_SRC_TOKEN] + EXTRA_TOKENS})
model.resize_token_embeddings(len(tokenizer))

# Force English output
model.config.forced_bos_token_id = tokenizer.convert_tokens_to_ids(TGT_LANG)


# === 5. Preprocess ===

In [ ]:
def preprocess(examples):
    src_texts = [f"{SPECIAL_SRC_TOKEN} {text}" for text in examples["src_text"]]

    model_inputs = tokenizer(
        src_texts,
        truncation=True,
        padding="max_length",
        max_length=128
    )

    labels = tokenizer(
        examples["tgt_text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )["input_ids"]

    labels = [[(t if t != tokenizer.pad_token_id else -100) for t in label] for label in labels]
    model_inputs["labels"] = labels

    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=True)

# === 6. Training Args ===

In [ ]:
metric_bleu = evaluate.load("sacrebleu")


def compute_metrics(eval_pred):
    preds, labels = eval_pred
    # preds are already token IDs because of generate()
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metric_bleu.compute(
        predictions=decoded_preds,
        references=[[l] for l in decoded_labels]
    )
    return {"sacrebleu": result["score"]}


training_args = Seq2SeqTrainingArguments(
    output_dir=MODEL_DIR,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    warmup_steps=200,
    logging_steps=50,
    eval_steps=200,
    save_steps=200,
    save_total_limit=3,
    predict_with_generate=True,
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),
)


# === 7. Trainer ===

In [ ]:
# === Fix 3: Data collator ===
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# === 8. Train ===

In [ ]:
trainer.train()

# === 9. Save ===

In [ ]:
trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)
print(f"✅ Model saved to: {MODEL_DIR}")

# === 10. Translation Function ===

In [ ]:
def batch_translate(texts, batch_size=8):
    results = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    for i in range(0, len(texts), batch_size):
        src_texts = [f"{SPECIAL_SRC_TOKEN} {t}" for t in texts[i:i+batch_size]]
        inputs = tokenizer(src_texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs)
        results.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
    return results

# === 11. Evaluate BLEU ===

In [ ]:
print("\n--- Evaluating BLEU Score ---")
bleu = evaluate.load("bleu")

test_df = dataset["test"].to_pandas()
preds = batch_translate(test_df["src_text"].tolist())
refs = [[x] for x in test_df["tgt_text"].tolist()]

bleu_score = bleu.compute(predictions=preds, references=refs)
print(" BLEU Score:", bleu_score)

# === 12. Manual Test ===

In [ ]:
print("\n--- Manual Test ---")
sample_texts = [
    "Ali ku balu",
    "Anya ka?",
    "Masanting ya ing panaun ngeni",
    "E ku makanyan",
]

for i, kap_text in enumerate(sample_texts):
    translated = batch_translate([kap_text])[0]
    print(f"[{i+1}] Kapampangan: {kap_text}")
    print(f"    ➤ English: {translated}")